In [1]:
remotes::install_github("ropensci/rmangal")

Skipping install of 'rmangal' from a github remote, the SHA1 (87df6fd8) has not changed since last install.
  Use `force = TRUE` to force installation



In [2]:
library(tidyverse)
library(magrittr)
library(dplyr)
library(sf)
library(rmangal)
library(raster)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract


Linking to GEOS 3.10.2, GDAL 3.5.0, PROJ 9.0.0; sf_use_s2() is TRUE

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select




In [52]:
networks_metadata_path = "../../data/networks/all/networks_metadata.csv"

rmangal_observations_data_path = "../../data/networks/metadata/parsed_rmangal_networks_metadata.csv"
rmangal_metadata_path = "../../data/networks/metadata/all_rmangal_networks.csv"

ecoregion_polygon_data_path1 = "../../data/networks/metadata/polygon/WWF/wwf_terr_ecos.shp"
ecoregion_polygon_data_path2 = "../../data/networks/metadata/polygon/Ecoregions2017/Ecoregions2017.shp"

temp_input_for_bioclim_path = "../../data/networks/all/geo_avail_networks_metadata.csv"
temp_output_for_bioclim_path = "../../data/networks/all/geo_extracted_networks_metadata.csv"
bioclim_dir = "../../data/networks/metadata/bioclimate/"
richness_data_path = "../../data/networks/metadata/Richness_2021.tif"
rmangal_geodata_path = "../../data/networks/rmangal_networks_geodata.csv"

# get geodata for networks

## Mangal

In [11]:
if (!file.exists(rmangal_metadata_path))
{
    pollination_interactions<- search_interactions(type = "mutualism")
    network_ids = unique(pollination_interactions$network_id)
    length(network_ids)
    
    all_networks_data = data.frame()
    network_metadata_dfs = list()
    num_avail = 0
    for (i in 1:length(network_ids)) #length(pollination_networks)
    {
      if (i %% 10 == 0) 
      {
        print(i)
        print(num_avail)
      }
      networks = search_networks(list(id=nth(network_ids,i)))  %>% get_collection
      network = networks[[1]]
      network_data = network$network[c("geom_lat", "geom_type", "geom_lon")]
      network_ref_data = network$reference
      network_data = bind_cols(network_data, network_ref_data)
      network_data[["network_id"]] = nth(network_ids,i)
      network_metadata_dfs[[i]] = network_data
      if (! is.na(network_data$geom_lon))
      {
        num_avail = num_avail + 1
      }
    df = ldply(network_metadata_dfs, rbind)
    df <- apply(df,2,as.character)
    write.csv(df,rmangal_metadata_path)
    }
}

In [5]:
rmangal_metadata = read.csv(rmangal_metadata_path)

In [8]:
head(rmangal_metadata[is.na(rmangal_metadata$geom_lat), ][c("network_id", "doi", "paper_url", "geom_lon", "geom_lat")])

,network_id,doi,paper_url,geom_lon,geom_lat
,<int>,<chr>,<chr>,<chr>,<chr>
6,1419,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
9,952,NA,null,NA,NA
10,1420,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
19,1310,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
20,1311,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA
21,1307,10.1111/ele.12821,https://onlinelibrary.wiley.com/doi/full/10.1111/ele.12821,NA,NA


In [ ]:
# merge rmangal_metadata to networks_metadata

# Web of life

In [88]:
base_url <- "https://www.web-of-life.es/"
networks_metadata = read.csv(networks_metadata_path)
non_wol_networks_metadata = networks_metadata[networks_metadata$source != "web_of_life", ] 
wol_networks_metadata = networks_metadata[networks_metadata$source == "web_of_life", ][c('network_type','network_index','path','source','is_legal','is_resolved','processed_path','is_parsed_legal')]                                                                                      
all_nw_info <- read.csv(paste0(base_url,"get_network_info.php"))

In [89]:
get_name <- function(path)
{
    path_name = basename(path)
    name = unlist(strsplit(path_name, ".", fixed=TRUE))[1]
    return (name)
}

In [90]:
wol_networks_metadata$network_name = unlist(lapply(wol_networks_metadata$path,get_name))

In [91]:
all_nw_info$Latitude <- all_nw_info$latitude
all_nw_info$Longitude <- all_nw_info$longitude

drops <- c("network_type","number_components", "is_weighted", "cell_values_description", "abundance_description", "latitude", "longitude")
all_nw_info = all_nw_info[ , !(names(all_nw_info) %in% drops)]

In [92]:
colnames(all_nw_info)

[1] "network_name"     "author"           "reference"        "location"        
[5] "location_address" "country"          "Latitude"         "Longitude"

In [93]:
wol_networks_metadata <- left_join(wol_networks_metadata, all_nw_info, by = join_by(network_name == network_name))

In [94]:
rest_networks_metdata =  networks_metadata[networks_metadata$source != "web_of_life", ]

In [104]:
networks_metadata = bind_rows(wol_networks_metadata, rest_networks_metdata)

In [106]:
write.csv(networks_metadata, networks_metadata_path)

# get bioclimate data for networks

In [34]:
networks_metadata = read.csv(networks_metadata_path)
networks_metadata = networks_metadata %>% filter(!is.na(Latitude))
networks_metadata = networks_metadata %>% filter(!is.na(Longitude))

In [35]:
networks_metadata["ID"] = c(1:dim(networks_metadata)[1])

In [6]:
write.csv(relevant_network_metadata, temp_input_for_bioclim_path)

In [38]:
networks_metadata[1, c('path','Latitude','Longitude')]$Latitude

[1] ""

## run the following via windows (does not work in ppn conda env)

In [53]:
# Load the data -----------------------------------------------------------
#--#Observation points (NEED columns "ID", "Latitude", "Longitude") #--# 
Observations = read_csv(networks_metadata_path)

#--#Ecoregion Polygon #--# 
Ecoregion_Polygon_1 = st_read(ecoregion_polygon_data_path1)
Ecoregion_Polygon_2 = st_read(ecoregion_polygon_data_path2)

New names:
• `` -> `...1`
Rows: 157 Columns: 67
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (15): path, processed_path, network_type, source, doi, paper_url, data_...
dbl  (38): ...1, Unnamed: 0.1, Unnamed: 0, network_index, EcoRegion, Annual ...
lgl  (12): reference, location_address, country, is_legal, is_resolved, is_p...
dttm  (2): created_at, updated_at

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Reading layer `wwf_terr_ecos' from data source 
  `/groups/itay_mayrose/halabikeren/plant_pollinator_networks/data/networks/metadata/polygon/WWF/wwf_terr_ecos.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 14458 features and 21 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -180 ymin: -89.89197 xmax: 180 ymax: 83.62313
Geodetic CRS:  WGS 84
Reading layer `Ecoregions2017' from data source 
  `/groups/itay_mayrose/halabikeren/plant_pollinator_networks/data/networks/metadata/polygon/Ecoregions2017/Ecoregions2017.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 847 features and 15 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -180 ymin: -89.89197 xmax: 180 ymax: 83.62313
Geodetic CRS:  WGS 84


In [54]:
#--#BioClim Data #--# 
my_path = bioclim_dir
BioClim_list = list.files(path = my_path, 
                          pattern = ".tif$", 
                          all.files = TRUE, 
                          full.names = FALSE)

In [55]:
#Load rasters (Should create a list of 19)
BioClim_Rasters = lapply(paste(my_path, 
                               BioClim_list,
                               sep = ""),
                         raster)
  
#--#Remover superfluous data #--# 
rm(my_path, BioClim_list)

In [56]:
#--#Regional Species Richness Data #--# 
Richness_Rasters = raster(richness_data_path)

#To check that the rest of the polygons are good, you can do the following
which(st_is_valid(Ecoregion_Polygon_1) != TRUE)

[1] 1526

In [57]:
bad_polygons_2_vec = -1*c(which(st_is_valid(Ecoregion_Polygon_2) != TRUE))

In [58]:
# Prepare the Ecoregion Data ----------------------------------------------
#For whatever region, the polygon in row 1526 is not good and will cause the whole thing to mess up.
#But since your data probably doesn't care about this you can just remove it 
#Info about the bad polygon:
  #OBJECT_ID = 1501
  #ECO_ID = 60125
  #ECO_NAME = Guianan moist forests
  #You can see the location of the region here: https://dopa-explorer.jrc.ec.europa.eu/ecoregion/60125
#Remove the bad polygon
Ecoregion_Polygon_1 = Ecoregion_Polygon_1[-1526, ]

#To check that the rest of the polygons are good, you can do the following
which(st_is_valid(Ecoregion_Polygon_1) != TRUE)

integer(0)

In [59]:
Ecoregion_Polygon_2 = Ecoregion_Polygon_2[bad_polygons_2_vec, ]
                                          
#To check that the rest of the polygons are good, you can do the following
which(st_is_valid(Ecoregion_Polygon_2) != TRUE)

integer(0)

In [60]:
# Prepare Observation Points ----------------------------------------------
#Convert the Observation data to an SF object
Observation_SF = Observations %>%
                   st_as_sf(coords = c("Longitude", 
                                       "Latitude"))

Warning message in lapply(x[coords], as.numeric):
“NAs introduced by coercion”
Warning message in lapply(x[coords], as.numeric):
“NAs introduced by coercion”


ERROR: Error in st_as_sf.data.frame(., coords = c("Longitude", "Latitude")): missing values in coordinates not allowed


In [49]:
Observations[, c("path", "Longitude", "Latitude")]

path,Longitude,Latitude
<chr>,<chr>,<chr>
../../data/networks//mangal/weighted/5129.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"
../../data/networks//mangal/weighted/1598.csv,-16.633333,28.216667
../../data/networks//mangal/weighted/5175.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"
../../data/networks//mangal/weighted/5015.csv,3.177,42.352
../../data/networks//mangal/weighted/5149.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"
../../data/networks//mangal/weighted/5148.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"
../../data/networks//mangal/weighted/5174.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"
../../data/networks//mangal/weighted/5176.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"
../../data/networks//mangal/weighted/5189.csv,"c(-106.252863, -106.252863, -107.9038794, -107.9038794, -106.252863)","c(39.150357, 37.933364, 37.933364, 39.150357, 39.150357)"


In [ ]:
#Give the observations the coordinate system it is referenced in (#This is dataset specific, if from GBIF use this number)
st_crs(Observation_SF) = 4326 
  
#Set the coordinate system of the observations to be the same as the Polygona
Observation_SF_1 = st_transform(Observation_SF,
                          crs = st_crs(Ecoregion_Polygon_1))

Observation_SF_2 = st_transform(Observation_SF,
                          crs = st_crs(Ecoregion_Polygon_2))

In [ ]:
# Intersect Points and Polygon --------------------------------------------
#NOTE THAT DEPENDING ON YOUR DATA NOT ALL POINTS MAY INTERSECT WITH AN ECOREGION
#Determine which polygon each point is in
sf_use_s2(FALSE)
Intersection_Data = as.data.frame(st_intersects(Observation_SF, Ecoregion_Polygon))

#Convert the row.ID to the observation ID
Intersection_Data$Obs_ID = Observations$ID[Intersection_Data$row.id]
  
#Convert the col.ID to the ecoregion ID
Intersection_Data$EcoRegion = Ecoregion_Polygon$ECO_ID[Intersection_Data$col.id]

In [ ]:
# Intersect Points and BioClim Rasters ------------------------------------
#Determine the raster values of each point (I do this individually to make sure no errors occur)

#Create a holder
BioClim_data = as.data.frame(matrix(nrow = nrow(Observations),
                                    ncol = length(BioClim_Rasters) +1))

#Set the column names
colnames(BioClim_data) = c("Obs_ID",
                            paste("BioClim_",
                                1:19,
                                sep = ""))
  
#Set the observation ID
BioClim_data$Obs_ID = Observations$ID
  
#Go through each raster
for (i in 1:length(BioClim_Rasters)) 
{
  #Extract values and place in holder
  BioClim_data[, i+1] = extract(BioClim_Rasters[[1]], Observation_SF)
}
  

#Add the data to the Intersection_Data for those whose ID match
Complete_Data = merge(Intersection_Data[, c("Obs_ID", "EcoRegion")], 
                      BioClim_data,
                      by.x = "Obs_ID",
                      by.y = "Obs_ID")
```

```{r}
#Create a holder
richness_data = as.data.frame(matrix(nrow = nrow(Observations),
                                    ncol = length(Richness_Rasters) +1))

#Set the column names
colnames(richness_data) = c("Obs_ID",
                            "species_richness")

#Set the observation ID
richness_data$Obs_ID = Observations$ID
  
#Go through each raster
richness_data[["species_richness"]] = extract(Richness_Rasters, Observation_SF)
  

#Add the data to the Intersection_Data for those whose ID match
Richness_Complete_Data = merge(Intersection_Data[, c("Obs_ID", "EcoRegion")], 
                      richness_data,
                      by.x = "Obs_ID",
                      by.y = "Obs_ID")
Richness_Complete_Data <- Richness_Complete_Data[c("Obs_ID", "EcoRegion", "species_richness")]

In [135]:
library(tidyverse)
full_data = merge(Complete_Data, 
                  Richness_Complete_Data,
                  by.x = "Obs_ID",
                  by.y = "Obs_ID")
write_csv(full_data, temp_output_for_bioclim_path)

## back in linux - unite data from out_temp_input_for_bioclim_path back to networks metadata

In [147]:
bioclim_out_data = read_csv(temp_output_for_bioclim_path)

Rows: 384 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (23): Obs_ID, EcoRegion.x, BioClim_1, BioClim_2, BioClim_3, BioClim_4, B...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [149]:
relevant_network_metadata = merge(relevant_network_metadata, bioclim_out_data, by.x='ID', by.y = "Obs_ID", all.x=TRUE)

In [162]:
drop = c("X", "ID", "network_name")
relevant_network_metadata = relevant_network_metadata[, !(names(relevant_network_metadata) %in% drops)]

In [161]:
rest_networks_metadata = networks_metadata[is.na(networks_metadata$Latitude) | is.na(networks_metadata$Longitude), ]
drop = c("X", "ID", "network_name")
rest_networks_metadata = rest_networks_metadata[ , !(names(rest_networks_metadata) %in% drops)]

In [167]:
networks_metadata = bind_rows(relevant_network_metadata, rest_networks_metadata)

In [170]:
write.csv(networks_metadata, networks_metadata_path)